In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Ridge

from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import classification_report
import warnings

# 특정 경고 메시지 무시
warnings.filterwarnings("ignore")

In [2]:
# data = pd.read_excel(r'C:\apps\ml\project1\datasets\filtered_features.xlsx')
data = pd.read_excel('filtered_features.xlsx')

### 날짜 datatime 형식으로 변환
data['날짜'] = pd.to_datetime(data['날짜'])

### 피쳐 정보 확인
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117 entries, 0 to 116
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   날짜              117 non-null    datetime64[ns]
 1   경상수지            117 non-null    float64       
 2   고용률             117 non-null    float64       
 3   WTI유가           117 non-null    float64       
 4   건설기성액(십억원)      117 non-null    float64       
 5   국내설비투자지수        117 non-null    float64       
 6   국내소매판매액지수       117 non-null    float64       
 7   미국ISM(제조업물가지수)  117 non-null    float64       
 8   미국취업자수증감(만명)    117 non-null    float64       
 9   미국생산자물가지수       117 non-null    float64       
 10  종합주택매매가격지수      117 non-null    float64       
 11  소비자물가지수         117 non-null    float64       
 12  소비자심리지수(CCSI)   117 non-null    float64       
 13  실업급여지급건수(월)     117 non-null    float64       
 14  전체임금총액(원)       117 non-null    float64       
 15  label 

In [12]:
data

,날짜,경상수지,고용률,WTI유가,건설기성액(십억원),국내설비투자지수,국내소매판매액지수,미국ISM(제조업물가지수),미국취업자수증감(만명),미국생산자물가지수,종합주택매매가격지수,소비자물가지수,소비자심리지수(CCSI),실업급여지급건수(월),전체임금총액(원),label
0,2013-04-01,-0.252263,0.006734,-0.0388,0.040479,-0.015169,0.000000,0.049123,0.309859,-0.002571,0.000411,-0.001388,-0.024952,0.061858,-0.0053,0
1,2013-05-01,0.906427,0.000000,-0.0159,0.010696,0.009479,0.007203,-0.113712,0.150538,0.000000,0.001778,0.000000,0.024606,-0.047892,-0.0323,0
2,2013-06-01,-0.289690,0.001672,0.0499,0.055571,0.030516,0.004768,-0.064151,-0.163551,-0.000515,0.000326,-0.001217,0.008646,-0.131500,0.0567,0
3,2013-07-01,0.020484,0.000000,0.0877,-0.019089,0.041002,0.009490,0.018145,-0.318436,0.000516,0.000229,0.002146,-0.002857,0.134213,0.0232,0
4,2013-08-01,-0.082603,0.000000,0.0249,0.012227,0.020788,-0.010576,0.073267,0.991803,0.003093,-0.000519,0.003541,0.001910,-0.088000,-0.0366,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112,2022-08-01,-3.216346,0.000000,-0.0920,-0.002917,0.069315,0.031369,-0.082716,-0.647826,-0.019616,-0.004939,-0.000920,0.032483,0.051012,-0.0555,1
113,2022-09-01,-1.453728,0.000000,-0.1123,-0.015550,-0.038602,-0.028571,-0.253028,0.049383,0.001579,-0.006241,0.001749,0.029213,-0.051524,0.1034,1
114,2022-10-01,-0.590460,0.000000,0.0886,0.019838,0.002273,0.014231,-0.064865,0.415686,0.008492,-0.007546,0.003124,-0.028384,-0.101469,-0.1109,1
115,2022-11-01,-2.736097,0.000000,-0.0690,0.004808,0.019652,-0.031805,0.011561,-0.285319,-0.004429,-0.013171,-0.000824,-0.026966,-0.012797,-0.0128,1


In [13]:
data['label'].value_counts()

label
0    60
1    57
Name: count, dtype: int64

In [14]:
# # 타겟 변수와 피처 분리
# X = data.drop(['날짜', 'label'], axis=1)
# y = data['label']

# 날짜 기준으로 데이터 분할
train_data = data[data['날짜'] <= pd.Timestamp('2020-12-31')]
test_data = data[data['날짜'] >= pd.Timestamp('2021-01-01')]
train_data.shape, test_data.shape

((93, 16), (24, 16))

In [15]:

# 타겟 변수와 피처 분리
X_train = train_data.drop(['날짜', 'label'], axis=1)
y_train = train_data['label']
X_test = test_data.drop(['날짜', 'label'], axis=1)
y_test = test_data['label']

In [16]:
y_train.value_counts()

label
0    55
1    38
Name: count, dtype: int64

In [17]:
# 0: 확장, 1: 수축
y_test.value_counts()

label
1    19
0     5
Name: count, dtype: int64

In [4]:
# 모델 생성
tree = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)
lgb = LGBMClassifier(random_state=42)
cat = CatBoostClassifier(random_state=42, verbose=0)

# 모델 학습
tree.fit(X_train, y_train)
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)
lgb.fit(X_train, y_train)
cat.fit(X_train, y_train)

# 테스트 세트로 예측
y_pred = tree.predict(X_test)
y_pred_rf = rf.predict(X_test)
y_pred_xgb = xgb.predict(X_test)
y_pred_lgb = lgb.predict(X_test)
y_pred_cat = cat.predict(X_test)

# 정확도 계산 및 기록
# acc_test = accuracy_score(y_test, y_pred)
# acc_test_list.append(acc_test)

[LightGBM] [Info] Number of positive: 38, number of negative: 55
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000372 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 441
[LightGBM] [Info] Number of data points in the train set: 93, number of used features: 14
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.408602 -> initscore=-0.369747
[LightGBM] [Info] Start training from score -0.369747
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best ga

In [18]:
help(precision_score)

Help on function precision_score in module sklearn.metrics._classification:

precision_score(y_true, y_pred, *, labels=None, pos_label=1, average='binary', sample_weight=None, zero_division='warn')
    Compute the precision.

    The precision is the ratio ``tp / (tp + fp)`` where ``tp`` is the number of
    true positives and ``fp`` the number of false positives. The precision is
    intuitively the ability of the classifier not to label as positive a sample
    that is negative.

    The best value is 1 and the worst value is 0.

    Read more in the :ref:`User Guide <precision_recall_f_measure_metrics>`.

    Parameters
    ----------
    y_true : 1d array-like, or label indicator array / sparse matrix
        Ground truth (correct) target values.

    y_pred : 1d array-like, or label indicator array / sparse matrix
        Estimated targets as returned by a classifier.

    labels : array-like, default=None
        The set of labels to include when ``average != 'binary'``, and thei

In [19]:
classification_report(y_test, y_pred)

'              precision    recall  f1-score   support\n\n           0       0.21      0.60      0.32         5\n           1       0.80      0.42      0.55        19\n\n    accuracy                           0.46        24\n   macro avg       0.51      0.51      0.43        24\nweighted avg       0.68      0.46      0.50        24\n'

In [20]:
print("DecisionTreeClassifier 성능:")
print(f"정확도: {accuracy_score(y_test, y_pred):.2f}")
print(f"정밀도: {precision_score(y_test, y_pred):.2f}")
print(f"재현율: {recall_score(y_test, y_pred):.2f}")
print(f"F1 점수: {f1_score(y_test, y_pred):.2f}")
print(classification_report(y_test, y_pred))
feature_importances_dt = pd.DataFrame(tree.feature_importances_,
                                      index = X_train.columns,
                                      columns=['importance']).sort_values('importance', ascending=False)

feature_importances_dt.reset_index(inplace=True)
feature_importances_dt.columns = ['feature', 'importance']
feature_importances_dt

DecisionTreeClassifier 성능:
정확도: 0.46
정밀도: 0.80
재현율: 0.42
F1 점수: 0.55
              precision    recall  f1-score   support

           0       0.21      0.60      0.32         5
           1       0.80      0.42      0.55        19

    accuracy                           0.46        24
   macro avg       0.51      0.51      0.43        24
weighted avg       0.68      0.46      0.50        24



,feature,importance
0,종합주택매매가격지수,0.390593
1,미국취업자수증감(만명),0.241139
2,건설기성액(십억원),0.141063
3,소비자물가지수,0.078798
4,WTI유가,0.049909
5,미국생산자물가지수,0.041531
6,고용률,0.038141
7,미국ISM(제조업물가지수),0.018826
8,경상수지,0.000000
9,국내설비투자지수,0.000000


In [23]:
y_pred_rf_train = rf.predict(X_train)

In [24]:
print("RandomForestClassifier 성능:")
print(f"정확도: {accuracy_score(y_train, y_pred_rf_train):.2f}")
print(f"정밀도: {precision_score(y_train, y_pred_rf_train):.2f}")
print(f"재현율: {recall_score(y_train, y_pred_rf_train):.2f}")
print(f"F1 점수: {f1_score(y_train, y_pred_rf_train):.2f}")

RandomForestClassifier 성능:
정확도: 1.00
정밀도: 1.00
재현율: 1.00
F1 점수: 1.00


In [22]:
print("RandomForestClassifier 성능:")
print(f"정확도: {accuracy_score(y_test, y_pred_rf):.2f}")
print(f"정밀도: {precision_score(y_test, y_pred_rf):.2f}")
print(f"재현율: {recall_score(y_test, y_pred_rf):.2f}")
print(f"F1 점수: {f1_score(y_test, y_pred_rf):.2f}")
print(classification_report(y_test, y_pred_rf))
feature_importances_rf = pd.DataFrame(rf.feature_importances_,
                                      index = X_train.columns,
                                      columns=['importance']).sort_values('importance', ascending=False)

feature_importances_rf.reset_index(inplace=True)
feature_importances_rf.columns = ['feature', 'importance']
feature_importances_rf

RandomForestClassifier 성능:
정확도: 0.62
정밀도: 1.00
재현율: 0.53
F1 점수: 0.69
              precision    recall  f1-score   support

           0       0.36      1.00      0.53         5
           1       1.00      0.53      0.69        19

    accuracy                           0.62        24
   macro avg       0.68      0.76      0.61        24
weighted avg       0.87      0.62      0.66        24



,feature,importance
0,종합주택매매가격지수,0.139016
1,고용률,0.086504
2,건설기성액(십억원),0.077392
3,소비자심리지수(CCSI),0.071107
4,국내설비투자지수,0.070081
5,미국생산자물가지수,0.068810
6,WTI유가,0.067700
7,전체임금총액(원),0.066534
8,소비자물가지수,0.063827
9,실업급여지급건수(월),0.062357


In [7]:
# y_pred_rf = lgb.predict(X_test)
print("LGBMClassifier 성능:")
print(f"정확도: {accuracy_score(y_test, y_pred_lgb):.2f}")
print(f"정밀도: {precision_score(y_test, y_pred_lgb, average='macro'):.2f}")
print(f"재현율: {recall_score(y_test, y_pred_lgb, average='macro'):.2f}")
print(f"F1 점수: {f1_score(y_test, y_pred_lgb, average='macro'):.2f}")
print(classification_report(y_test, y_pred_lgb))
feature_importances_lgb = pd.DataFrame(lgb.feature_importances_,
                                      index = X_train.columns,
                                      columns=['importance']).sort_values('importance', ascending=False)
pd.DataFrame(feature_importances_lgb)

LGBMClassifier 성능:
정확도: 0.62
정밀도: 0.68
재현율: 0.76
F1 점수: 0.61
              precision    recall  f1-score   support

           0       0.36      1.00      0.53         5
           1       1.00      0.53      0.69        19

    accuracy                           0.62        24
   macro avg       0.68      0.76      0.61        24
weighted avg       0.87      0.62      0.66        24



,importance
건설기성액(십억원),52
소비자물가지수,35
고용률,27
WTI유가,25
경상수지,24
미국생산자물가지수,19
전체임금총액(원),17
종합주택매매가격지수,15
국내설비투자지수,12
소비자심리지수(CCSI),12


In [8]:
print("XGBClassifier 성능:")
print(f"정확도: {accuracy_score(y_test, y_pred_xgb):.2f}")
print(f"정밀도: {precision_score(y_test, y_pred_xgb, average='macro'):.2f}")
print(f"재현율: {recall_score(y_test, y_pred_xgb, average='macro'):.2f}")
print(f"F1 점수: {f1_score(y_test, y_pred_xgb, average='macro'):.2f}")
print(classification_report(y_test, y_pred_xgb))
feature_importances_xgb = pd.DataFrame(xgb.feature_importances_,
                                      index = X_train.columns,
                                      columns=['importance']).sort_values('importance', ascending=False)
feature_importances_xgb.reset_index(inplace=True)
feature_importances_xgb.columns = ['feature', 'importance']
feature_importances_xgb

XGBClassifier 성능:
정확도: 0.58
정밀도: 0.67
재현율: 0.74
F1 점수: 0.57
              precision    recall  f1-score   support

           0       0.33      1.00      0.50         5
           1       1.00      0.47      0.64        19

    accuracy                           0.58        24
   macro avg       0.67      0.74      0.57        24
weighted avg       0.86      0.58      0.61        24



,feature,importance
0,종합주택매매가격지수,0.211416
1,국내소매판매액지수,0.148273
2,미국생산자물가지수,0.083075
3,고용률,0.073357
4,건설기성액(십억원),0.068066
5,전체임금총액(원),0.066103
6,국내설비투자지수,0.055360
7,소비자심리지수(CCSI),0.051451
8,경상수지,0.050560
9,실업급여지급건수(월),0.045833


In [26]:
feature_importances_xgb['importance'].sum()

0.9999999

In [9]:
print("CatBoostClassifier 성능:")
print(f"정확도: {accuracy_score(y_test, y_pred_cat):.2f}")
print(f"정밀도: {precision_score(y_test, y_pred_cat, average='macro'):.2f}")
print(f"재현율: {recall_score(y_test, y_pred_cat, average='macro'):.2f}")
print(f"F1 점수: {f1_score(y_test, y_pred_cat, average='macro'):.2f}")
print(classification_report(y_test, y_pred_cat))
feature_importances_cat = pd.DataFrame(cat.feature_importances_,
                                      index = X_train.columns,
                                      columns=['importance']).sort_values('importance', ascending=False)
pd.DataFrame(feature_importances_cat)

CatBoostClassifier 성능:
정확도: 0.54
정밀도: 0.66
재현율: 0.71
F1 점수: 0.53
              precision    recall  f1-score   support

           0       0.31      1.00      0.48         5
           1       1.00      0.42      0.59        19

    accuracy                           0.54        24
   macro avg       0.66      0.71      0.53        24
weighted avg       0.86      0.54      0.57        24



,importance
종합주택매매가격지수,13.717705
건설기성액(십억원),9.912086
고용률,8.419037
국내설비투자지수,7.758776
미국생산자물가지수,7.032058
경상수지,6.938465
미국취업자수증감(만명),6.538445
전체임금총액(원),6.116751
국내소매판매액지수,6.090496
소비자물가지수,5.943642


In [10]:
# 데이터 표준화
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)